In [1]:
!pip3 install -e ..

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///data/pathology/users/thijs/nnunet-for-pathology
  Preparing metadata (setup.py) ... done
  Attempting uninstall: nnunet-pathology
    Found existing installation: nnunet-pathology 0.0.0
    Uninstalling nnunet-pathology-0.0.0:
      Successfully uninstalled nnunet-pathology-0.0.0
  Running setup.py develop for nnunet-pathology


In [3]:
!pip3 install git+https://github.com/DIAGNijmegen/pathology-whole-slide-data@8a0e97b22dcf503cc8325546d5812fb4e9c3df50

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/DIAGNijmegen/pathology-whole-slide-data (to revision 8a0e97b22dcf503cc8325546d5812fb4e9c3df50) to /tmp/pip-req-build-3gh_269w
  Running command git clone --filter=blob:none --quiet https://github.com/DIAGNijmegen/pathology-whole-slide-data /tmp/pip-req-build-3gh_269w
  Running command git rev-parse -q --verify 'sha^8a0e97b22dcf503cc8325546d5812fb4e9c3df50'
  Running command git fetch -q https://github.com/DIAGNijmegen/pathology-whole-slide-data 8a0e97b22dcf503cc8325546d5812fb4e9c3df50
  Running command git checkout -q 8a0e97b22dcf503cc8325546d5812fb4e9c3df50
  Resolved https://github.com/DIAGNijmegen/pathology-whole-slide-data to commit 8a0e97b22dcf503cc8325546d5812fb4e9c3df50
  Preparing metadata (setup.py) ... done
  Created wheel for wholeslidedata: filename=wholeslidedata-0.0.13-py3-none-any.whl size=79157 sha256=2462b767db0cd4ac80247f5ee5abb7e373375216f00b775f9db2ab4b929ec693

In [1]:
from nnunet_pathology.datamodules.wsi_datamodule import WholeSlideDataModule
import numpy as np
from tqdm.notebook import tqdm

In [15]:
def get_mean_and_std(dataloader):
    channels_sum, channels_squared_sum, num_batches = 0, 0, 0
    for data, _ in tqdm(dataloader):
        # Mean over batch, height and width, but not over the channels
        channels_sum += data.mean(axis=(0,1,2))
        channels_squared_sum += (data**2).mean(axis=(0,1,2))
        num_batches += 1
    
    mean = channels_sum / num_batches

    # std = sqrt(E[X^2] - (E[X])^2)
    std = (channels_squared_sum / num_batches - mean ** 2) ** 0.5

    return mean, std

In [16]:
n_samples = 1000

datamodule = WholeSlideDataModule(
    user_train_config = "/data/pathology/projects/pathology-endoaid/nnunet-for-pathology/train_config.yml",
    user_val_config = "/data/pathology/projects/pathology-endoaid/nnunet-for-pathology/valid_config.yml",
    context = "fork",
    steps_per_epoch = n_samples,
    num_classes = 6,
    num_workers = 6
)

datamodule.setup()

dataloader = iter(datamodule.train_dataloader())

print(get_mean_and_std(dataloader))

  0%|          | 0/1000 [00:00<?, ?it/s]

(tensor([194.6097, 156.8905, 179.4379]), tensor([55.3238, 64.9420, 54.2014]))
